In [ ]:
import geopandas as gpd
import pandas as pd

# Caminho do shapefile
caminho = "data/PRODES_AMZ_Municipal_2023.shp"

# Ler o shapefile
gdf = gpd.read_file(caminho)

print("Colunas encontradas no shapefile:")
print(gdf.columns)


# Agrupar por estado e ano, somando área total desmatada
# Salvar como CSV

gdf.to_csv('data/desmatamento_por_estado.csv', index=False)

print("✅ Arquivo CSV agregado salvo com sucesso em data/desmatamento_por_estado.csv")



In [ ]:
import pandas as pd

url = "https://apisidra.ibge.gov.br/values/t/5938/n6/all/v/37/p/2021"
df = pd.read_json(url)
df.to_csv("data/pib_municipal_2023.csv", index=False)

print("✅ PIB municipal 2021 salvo com sucesso!")
